<a href="https://colab.research.google.com/github/Ali-Khatib/AI-Pet-Breed-Classifier/blob/main/LLM_Based_Email_Summarization_and_Response_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U \
  langchain \
  langchain-community \
  langchain-core \
  langchain-text-splitters \
  langchain-groq \
  transformers \
  accelerate \
  sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 613.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
  Attempting uninstall: lang

## IMPORTS

In [ ]:
import os
import re
import torch
import warnings
warnings.filterwarnings("ignore")


from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence


from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import TextLoader

from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter


print("IMPORTS ARE READY :D")


IMPORTS ARE READY :D


## Integrating Groq with Meta LLaMA 3.1

In [ ]:
os.environ["GROQ_API_KEY"] = "Prof please go to  https://console.groq.com/keys and create a key its free and its what we used"

prompt="Introduce yourself and say hi."


llm = ChatGroq(model="llama-3.1-8b-instant")
response = llm.invoke(prompt)
print(response.content)


Hello, I'm an AI assistant, which means I'm a computer program designed to understand and respond to human language. I don't have a personal name, but I'm here to help answer your questions, provide information, and chat with you on a wide range of topics.

I'm constantly learning and improving, so I may not always have the latest information or the most accurate answers. However, I'll do my best to provide helpful and informed responses.

So, hi there! It's great to meet you. Is there something I can help you with or would you like to chat about something in particular?


## User Input Handling for Email Content


In [ ]:
choice = input(
    "Choose input method:\n"
    "1 - Paste email manually\n"
    "2 - Upload a .txt file\n"
    "Enter 1 or 2: "
).strip()

if choice == "1":
    email_text = input("Please paste the email:\n").strip()

elif choice == "2":
    from google.colab import files
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    with open(file_name, "r", encoding="utf-8") as f:
        email_text = f.read()

else:
    raise ValueError("Invalid choice.")


Choose input method:
1 - Paste email manually
2 - Upload a .txt file
Enter 1 or 2: 2


Saving use_this.txt to use_this.txt


## Text Chunking for LLM Processing


In [ ]:
email_text = email_text.strip()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = splitter.split_text(email_text)


## Prompt Design and Chunk-Level Summarization


In [ ]:
summary_prompt = PromptTemplate.from_template(
    """
  Summarize the following text concisely.

  Rules:
  - Produce a short content summary, not a rewritten email.
  - Do NOT include greetings, closings, or subject lines.
  - Do NOT reorganize the text into action plans or proposals.
  - Preserve names if explicitly mentioned.

  IMPORTANT: Explicitly state what the sender expects or requests from the recipient in 1 sentence, if any.


  Text:
  {chunk}
  """
)

summarize_chain = summary_prompt | llm | StrOutputParser()

chunk_summaries = [summarize_chain.invoke({"chunk": c}) for c in chunks]


In [ ]:
final_summary_prompt = PromptTemplate.from_template(
    "Combine the following summaries into one clear summary:\n\n{summaries}"
)

final_summary_chain = final_summary_prompt | llm | StrOutputParser()

final_summary = final_summary_chain.invoke(
    {"summaries": "\n".join(chunk_summaries)}
)


In [ ]:
print(final_summary)

Here's a clear and comprehensive summary:

Jordan, the sender, is seeking to clarify the current status and potential next steps for Q3 deliverables due to confusion over priorities, timelines, and ownership. Key issues affecting the project include:

- The initial implementation of the analytics module was completed on schedule, but internal testing revealed edge cases requiring additional development time.
- Limited resources due to team member reassignments have impacted the team's work.
- Integration work with an external API provider is experiencing unexpected challenges due to recent updates to their authentication flow, causing intermittent failures in the staging environment.
- Uncertainty over the current integration approach, with either a workaround or a complete adjustment needed.
- Ongoing discussions about the reporting dashboard's scope, including additional visualizations and filters requested by stakeholders.
- The sender proposes improving project communication and co

## User-Defined Output Configuration Selection


In [ ]:
length_options = {
    "1": "short",
    "2": "medium",
    "3": "long"
}

tone_options = {
    "1": "formal",
    "2": "casual",
    "3": "polite"
}

print("\nChoose email length:")
print("1 - Short")
print("2 - Medium")
print("3 - Long")
length_choice = input("Enter 1, 2, or 3: ").strip()

print("\nChoose tone:")
print("1 - Formal")
print("2 - Casual")
print("3 - Polite")
tone_choice = input("Enter 1, 2, or 3: ").strip()



length = length_options.get(length_choice, "medium")
tone = tone_options.get(tone_choice, "formal")



Choose email length:
1 - Short
2 - Medium
3 - Long
Enter 1, 2, or 3: 1

Choose tone:
1 - Formal
2 - Casual
3 - Polite
Enter 1, 2, or 3: 3


In [ ]:
response_core = input(
    "Please briefly describe the main point your response should focus on:\n"
).strip()

Please briefly describe the main point your response should focus on:
Agree on the importance of meeting ASAP to discuss everything that has been suggested in the meeting :D


## Controlled Email Response Generation


In [ ]:
generation_prompt = PromptTemplate.from_template(
"""
You are writing a RESPONSE email.

Roles (must follow):
- The original email was written by the sender.
- You are the recipient responding to the sender.
- Write from the recipient’s perspective only.

Rules (must follow):
- Do NOT write from the sender’s perspective.
- Do NOT rewrite or summarize the original email.
- Respond ONLY to the information explicitly stated in the summary.
- Do NOT invent new context, requests, or background.
- If the summary contains no clear request or question, write a neutral acknowledgment.
- Preserve personal names exactly as they appear in the original email, if any.
- If names are present, address the sender correctly and sign as the recipient.
- If names are not present, write a generic but appropriate response.

User guidance for response focus:
{response_core}

Length:
{length}

Tone:
{tone}

Summary of the received email:
{summary}

Original email (for role clarity and name preservation only — do not rewrite):
{original_email}
"""
)


generation_chain = generation_prompt | llm | StrOutputParser()


response_email = generation_chain.invoke({
    "summary": final_summary,
    "original_email": email_text,
    "response_core": response_core,
    "length": length,
    "tone": tone
})



In [ ]:

print("\nGenerated Email:\n")
print(response_email)



Generated Email:

Subject: Re: Q3 Deliverables and Next Steps

Dear Jordan,

I completely agree on the importance of meeting ASAP to discuss everything that has been suggested in the meeting. I'm looking forward to collaborating with you to address the key issues and revise the Q3 timeline.

Best regards,
[Your Name]


## Saving and Exporting the Generated Email


In [ ]:
file_path = "generated_email_response.txt"

with open(file_path, "w", encoding="utf-8") as f:
    f.write(response_email)

print(f"Saved to {file_path}")


from google.colab import files
files.download(file_path)


Saved to generated_email_response.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>